In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('recipes_info.csv')
df.head()

,recipe_id,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
0,2,india,india,stove top / bowl food,curry,fish,basmati,rice,shellfish,prawns,fish & seafood,no,no spice,25
1,5,italy,italy,protein&veg,meat & side veg,meat,white potatoes,potato,beans,butter beans,pulses,no,mild,35
2,9,thailand,thailand,stove top / bowl food,curry,vegan,basmati,rice,tofu,protein not found,vegetarian,no,spicy,25
3,16,united kingdom,united kingdom,protein&veg,fish & side veg,fish,NaN,NaN,oily fish,trout,fish & seafood,no,no spice,20
4,19,tunisia,tunisia,protein&veg,egg dish,vegetarian,millet,grains,eggs,eggs,dairy & eggs,no,no spice,30


In [47]:
cols =  ['country', 'dish_category',
         'dish_type', 'diet_type', 'carbohydrate_base', 'carbohydrate_category',
        'protein', 'protein_cut', 'protein_type', 'family_friendly',
        'spice_level', 'prep_time']

In [57]:
base = df.copy()

In [58]:
base.index = base.recipe_id
del base['recipe_id']

# treat mising content as information
base.fillna('missing', inplace=True)
base.head(10)

,country,country_secondary,dish_category,dish_type,diet_type,carbohydrate_base,carbohydrate_category,protein,protein_cut,protein_type,family_friendly,spice_level,prep_time
recipe_id,,,,,,,,,,,,,
2,india,india,stove top / bowl food,curry,fish,basmati,rice,shellfish,prawns,fish & seafood,no,no spice,25
5,italy,italy,protein&veg,meat & side veg,meat,white potatoes,potato,beans,butter beans,pulses,no,mild,35
9,thailand,thailand,stove top / bowl food,curry,vegan,basmati,rice,tofu,protein not found,vegetarian,no,spicy,25
16,united kingdom,united kingdom,protein&veg,fish & side veg,fish,missing,missing,oily fish,trout,fish & seafood,no,no spice,20
19,tunisia,tunisia,protein&veg,egg dish,vegetarian,millet,grains,eggs,eggs,dairy & eggs,no,no spice,30
20,tunisia,tunisia,protein&veg,cheese & side veg,vegetarian,cous cous,grains,cheese,feta,dairy & eggs,no,no spice,40
30,italy,italy,stove top / bowl food,pasta,vegetarian,white pasta,pasta,cheese,cheddar,dairy & eggs,no,no spice,40
53,lebanon,lebanon,salads,tabbouleh,vegetarian,bulgur wheat,grains,cheese,halloumi,dairy & eggs,no,no spice,30
54,italy,italy,stove top / bowl food,risotto,vegetarian,risotto rice,rice,missing,missing,missing,no,no spice,40


In [59]:
def return_sim_scores(I, recipe_id):
    i = I.loc[recipe_id]
    
    intersection = (i == I).sum(axis=1)
    total_set = i.count()+I.count(axis=1)-intersection
    jaccard = intersection/total_set
    return pd.DataFrame(jaccard, columns=[recipe_id])
    
for i, recipe_id in enumerate(base.index[:10]):
    if i == 0:
        jaqqard_sim = return_sim_scores(base, recipe_id)
    else:
        next_recipe = return_sim_scores(base, recipe_id)
        jaqqard_sim = jaqqard_sim.merge(next_recipe, 
                                        left_index=True,
                                        right_index=True)

        
        
jaqqard_sim.head(10)

,2,5,9,16,19,20,30,53,54,57
recipe_id,,,,,,,,,,
2,1.000000,0.040000,0.300000,0.181818,0.083333,0.083333,0.130435,0.083333,0.181818,0.040000
5,0.040000,1.000000,0.040000,0.083333,0.083333,0.083333,0.130435,0.040000,0.130435,0.083333
9,0.300000,0.040000,1.000000,0.040000,0.040000,0.040000,0.083333,0.040000,0.130435,0.000000
16,0.181818,0.083333,0.040000,1.000000,0.130435,0.130435,0.083333,0.083333,0.083333,0.040000
19,0.083333,0.083333,0.040000,0.130435,1.000000,0.444444,0.181818,0.300000,0.130435,0.040000
20,0.083333,0.083333,0.040000,0.130435,0.444444,1.000000,0.300000,0.300000,0.181818,0.040000
30,0.130435,0.130435,0.083333,0.083333,0.181818,0.300000,1.000000,0.238095,0.368421,0.040000
53,0.083333,0.040000,0.040000,0.083333,0.300000,0.300000,0.238095,1.000000,0.130435,0.040000
54,0.181818,0.130435,0.130435,0.083333,0.130435,0.181818,0.368421,0.130435,1.000000,0.040000


In [66]:
jaqqard_sim[20].sort_values(ascending=False).head()

recipe_id
20     1.000000
338    0.529412
428    0.444444
359    0.444444
234    0.444444
Name: 20, dtype: float64

In [67]:
base.loc[20]

country                            tunisia
country_secondary                  tunisia
dish_category                  protein&veg
dish_type                cheese & side veg
diet_type                       vegetarian
carbohydrate_base                cous cous
carbohydrate_category               grains
protein                             cheese
protein_cut                           feta
protein_type                  dairy & eggs
family_friendly                         no
spice_level                       no spice
prep_time                               40
Name: 20, dtype: object

In [68]:
base.loc[338]

country                            morocco
country_secondary                  morocco
dish_category                  protein&veg
dish_type                cheese & side veg
diet_type                       vegetarian
carbohydrate_base             bulgur wheat
carbohydrate_category               grains
protein                             cheese
protein_cut                           feta
protein_type                  dairy & eggs
family_friendly                         no
spice_level                          spicy
prep_time                               40
Name: 338, dtype: object

In [5]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard

res = pdist(df[cols], 'jaccard')
squareform(res)
distance = pd.DataFrame(squareform(res), index=df.index, columns= df.index)

ValueError: could not convert string to float: 'Spicy'

In [ ]:
squareform(res)
distance = pd.DataFrame(squareform(res), index=df.index, columns= df.index)